# ACOS-ABSA: Unified Preprocessing Pipeline for ModernBERT

1. Il paper originale utilizza BERT-base-uncased (Devlin et al., 2018) come encoder principale. Sebbene rivoluzionario all'epoca, BERT ha un contesto limitato (512 token) e una capacità di generalizzazione inferiore rispetto ai modelli attuali.

    * **La Nostra Soluzione:** Utilizziamo ModernBERT-base, un modello State-of-the-Art (2024) addestrato su un corpus molto più vasto e con una finestra di contesto estesa (8k token).

    * **Vantaggio:** ModernBERT offre rappresentazioni contestuali più ricche, fondamentali per risolvere il problema degli Aspetti Impliciti (es. dedurre Price da "it is expensive"), che rappresentano una sfida critica nel dataset ACOS.
+1

2. Strategia di Allineamento Sub-word (Robust Tokenization)
Il paper gestisce l'allineamento tra parole e token, ma spesso i modelli standard soffrono quando una parola annotata (es. "difficulty") viene spezzata in più sub-token (diffic, ##ulty).

    * **La Nostra Soluzione:** Abbiamo implementato una pipeline di preprocessing personalizzata che utilizza i word_ids per mappare precisamente le etichette BIO sui sub-token.

    * **Vantaggio:** Questo garantisce che nessun'informazione venga persa durante la tokenizzazione: se una parola è un'Opinione, tutti i suoi frammenti (token) erediteranno correttamente l'etichetta, migliorando la Recall del modello.

3. Semplificazione Architetturale (Rimozione del CRF)
Il modello Extract-Classify-ACOS impiega un layer CRF (Conditional Random Field) sopra BERT per "pulire" la sequenza di tag predetti e imporre vincoli logici.

   * **La Nostra Soluzione:** Sfruttando la maggiore potenza di estrazione delle feature di ModernBERT, iniziamo con una Linear Classification Head standard.

   * **Vantaggio:** Questo riduce drasticamente la complessità computazionale e i tempi di addestramento/inferenza. La capacità superiore di ModernBERT di apprendere le dipendenze locali rende spesso superfluo l'uso di un CRF, permettendo al modello di apprendere i vincoli BIO direttamente dai dati.

4. Gestione "Native" degli Span Impliciti
Come evidenziato nel paper, una larga percentuale di quadruple contiene aspetti o opinioni implicite (Span Nulli).

    * **La Nostra Soluzione:** Il nostro preprocessing gestisce esplicitamente gli span (-1, -1) nel dataset, preparando il terreno per la fase successiva (Classificazione). Mentre la fase di estrazione corrente si concentra sugli span espliciti, i vettori [CLS] di ModernBERT sono già ottimizzati per catturare il contesto globale necessario a predire le categorie implicite nel secondo step della pipeline.

In [1]:
# Import delle librerie necessarie
import torch
import numpy as np
import random
import pandas as pd
import wandb
import os

print("Librerie caricate.")

Librerie caricate.


### Impostazioni per la riproducibilità 

In [2]:
def set_seed(seed_value=42):
    """Imposto i seed per la riproducibilità."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        # Imposto anche i seed per la GPU, se disponibile
        torch.cuda.manual_seed_all(seed_value)

# Esegui l'impostazione del seed
set_seed(42) 
print("Random seeds impostati su 42.")

Random seeds impostati su 42.


### Configurazione Iniziale degli Hyperparameters (W&B)

Questa sezione definisce i principali parametri di addestramento (*hyperparameters*) per il *fine-tuning* del modello **ModernBERT**, registrandoli in **Weights & Biases** per tracciabilità e riproducibilità.

| Parametro | Valore | Motivazione della Scelta |
| :--- | :--- | :--- |
| **`learning_rate`** | `5e-5` (0.00005) | È il tasso di apprendimento *default* consigliato da Google/Hugging Face per il *fine-tuning* dei modelli BERT/RoBERTa. È un valore conservativo che assicura che il modello si adatti al nuovo *task* (ACOS) senza "dimenticare" le conoscenze acquisite nel *pre-training*. |
| **`epochs`** | `5` | Un valore tipico e contenuto per il *fine-tuning* di modelli Transformer. Di solito, 3 o 4 *epochs* sono sufficienti per convergere, ma 5 offrono un buon equilibrio tra addestramento e prevenzione dell'overfitting sui dataset di dimensioni limitate come ACOS. |
| **`batch_size`** | `16` | Questa dimensione del *batch* è comune quando si lavora con modelli grandi come RoBERTa-base, bilanciando la stabilità dell'addestramento con le limitazioni della **memoria GPU**. |
| **`model_name`** | `answerdotai/ModernBERT-base` | Scegliamo questo modello specifico Encoder-only come "ModernBERT" per le prestazioni superiori e un pre-training più robusto rispetto al BERTbase originale |
| **`dataset`** | `Laptop-ACOS` | Identifica il sotto-dataset specifico utilizzato per questo esperimento. |
| **`seed`** | `42` | Il **seed di riproducibilità**, fissato a 42 (la convenzione standard ML), garantisce che ogni volta che lo *script* viene eseguito, l'inizializzazione dei pesi e lo *shuffling* dei dati siano identici, garantendo la tracciabilità scientifica dei risultati. |

---

In [3]:
WANDB_ENTITY = "cristinatextmining"

# 1. Definizione degli Hyperparameters
config = {
    "learning_rate": 5e-5,
    "epochs": 5,
    "batch_size": 16,
    "model_name": "answerdotai/ModernBERT-base",
    "dataset": "Laptop-ACOS",
    "seed": 42,
    'patience': 2  # Per Early Stopping
}

# 2. Inizializzazione del Run
wandb.init(
    project="BigData-TextMining-ACOS",
    entity=WANDB_ENTITY,
    config=config,
    name=f"run_{config['model_name']}_{config['dataset']}"
)

print(f"W&B inizializzato per il progetto: {wandb.run.project}")

wandb: Currently logged in as: cristinatomaciello2001 (cristinatextmining) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B inizializzato per il progetto: BigData-TextMining-ACOS


## Raw Data Parsing & Quadruple Decoding

### Caricamento dei 3 dataset LAPTOP-ACOS e dei 3 dataset RESTAURANT-ACOS
Questa cella è dedicata al caricamento dei dataset TSV di ACOS (Training, Development e Test) dalla directory locale.

Data la struttura complessa del file di annotazione, che contiene tabulazioni (\t) interne e un numero variabile di quadruple per riga, la funzione load_as_single_string adotta una strategia di caricamento flessibile:

1. **Forzatura Stringa Unica:** Viene utilizzato un separatore inesistente (\x07) per istruire Pandas a caricare l'intera riga TSV come una singola colonna stringa *(full_line_data).*

2. **Robustezza:** Questo approccio previene i comuni ParserError causati da tabulazioni o delimitatori sporchi interni, garantendo che i dati grezzi vengano letti completamente senza perdita.

L'output di questa cella sono i tre DataFrame *(df_train, df_dev, df_test)*, ciascuno pronto per il parsing sequenziale sul contenuto della colonna full_line_data.

In [4]:
import pandas as pd
import os
import sys

# Definisci il percorso per ACOS
DATA_DIR_ACOS = 'data/Laptop-ACOS/'

# Definisci i percorsi completi dei file
TRAIN_FILE_PATH_ACOS = os.path.join(DATA_DIR_ACOS, 'laptop_train_quad_bert.tsv')
DEV_FILE_PATH_ACOS = os.path.join(DATA_DIR_ACOS, 'laptop_dev_quad_bert.tsv')
TEST_FILE_PATH_ACOS = os.path.join(DATA_DIR_ACOS, 'laptop_test_quad_bert.tsv')

# Definisci il percorso REESTAURANT
DATA_DIR_RESTAURANT = 'data/Restaurant-ACOS/'

# Definisci i percorsi completi dei file
TRAIN_FILE_PATH_RESTAURANT = os.path.join(DATA_DIR_RESTAURANT, 'rest16_train_quad_bert.tsv')
DEV_FILE_PATH_RESTAURANT = os.path.join(DATA_DIR_RESTAURANT, 'rest16_dev_quad_bert.tsv')
TEST_FILE_PATH_RESTAURANT = os.path.join(DATA_DIR_RESTAURANT, 'rest16_test_quad_bert.tsv')




def load_as_single_string(path):
    """
    Carica l'intero file TSV in un'unica colonna stringa, forzando la lettura riga per riga, 
    per evitare errori di parsing dovuti a delimitatori interni.
    """
    # 1. Utilizza read_csv ma forza l'uso di un separatore inesistente e non usare header
    df = pd.read_csv(
        path, 
        sep='\x07', # Separa per un carattere inesistente (Bell character)
        header=None, 
        on_bad_lines='skip', # Ignora le righe che danno problemi di formattazione
        engine='python'
    )
    
    # Rinomina l'unica colonna che contiene l'intera riga TSV
    df.columns = ['full_line_data']    
    print(f"File {path.split('/')[-1]} caricato come {df.shape[1]} colonna unica.")
    return df

# Esecuzione del caricamento corretto
try:
    df_train_laptop = load_as_single_string(TRAIN_FILE_PATH_ACOS)
    df_dev_laptop = load_as_single_string(DEV_FILE_PATH_ACOS)
    df_test_laptop = load_as_single_string(TEST_FILE_PATH_ACOS)
    
    df_train_rest = load_as_single_string(TRAIN_FILE_PATH_RESTAURANT)
    df_dev_rest = load_as_single_string(DEV_FILE_PATH_RESTAURANT)
    df_test_rest = load_as_single_string(TEST_FILE_PATH_RESTAURANT)
    
    print("Caricamento Flessibile Completato.")
    
except Exception as e:
    print(f"Errore critico durante il caricamento: {e}")
    raise

print(df_train_laptop.shape, df_dev_laptop.shape, df_test_laptop.shape)
print(df_train_rest.shape, df_dev_rest.shape, df_test_rest.shape)

File laptop_train_quad_bert.tsv caricato come 1 colonna unica.
File laptop_dev_quad_bert.tsv caricato come 1 colonna unica.
File laptop_test_quad_bert.tsv caricato come 1 colonna unica.
File rest16_train_quad_bert.tsv caricato come 1 colonna unica.
File rest16_dev_quad_bert.tsv caricato come 1 colonna unica.
File rest16_test_quad_bert.tsv caricato come 1 colonna unica.
Caricamento Flessibile Completato.
(2934, 1) (326, 1) (816, 1)
(1530, 1) (171, 1) (583, 1)


### Pipeline di Parsing e Strutturazione dei Dati(Quadruple)
Questa sezione implementa la pipeline di pre-elaborazione finale che trasforma la riga grezza di input in un formato Python strutturato, pronto per la Tokenizzazione del modello ModernBERT.

Il codice esegue un processo a due fasi per ogni riga del dataset:

1. **Separazione (Parsing della Riga Grezza)**: Esegue la divisione della riga unica *(full_line_data)* utilizzando il separatore Tab (\t). Questo isola la recensione pulita *(review_text)* dalla stringa contenente le quadruple codificate (il target).

2. **Decodifica Strutturale (Deep Parsing)**: Applica la funzione *parse_target_quadruples* alla stringa target. Questa funzione scompone la stringa in un dizionario con chiavi chiare **(span_A, span_B, category_aspect, sentiment)**, gestendo gli Indici Span Nullo (-1,-1) e validando la struttura a 4 elementi.

L'output finale è il DataFrame pulito con la colonna parsed_quadruples, che contiene le informazioni di target necessarie per creare le label di Sequence Labeling nella fase successiva.


In [5]:
import pandas as pd
import re
import numpy as np

# --- I. DEFINIZIONE DELLE FUNZIONI DI PARSING (Ottimizzate) ---

def parse_target_quadruples(target_string):
    """
    Decodifica la stringa di una singola quadrupla in un dizionario strutturato.
    """
    if not target_string:
        return {} 

    parts = target_string.split()
    if len(parts) != 4:
        return {}
    
    span_A_str, category_aspect, sentiment_str, span_B_str = parts

    def parse_span(span_str):
        if span_str == '-1,-1':
            return (-1, -1)
        try:
            start, end = map(int, span_str.split(','))
            return (start, end)
        except ValueError:
            return (-1, -1)

    span_A = parse_span(span_A_str)
    span_B = parse_span(span_B_str)
    
    try:
        sentiment = int(sentiment_str)
    except ValueError:
        sentiment = -1 

    return {
        'span_A': span_A,
        'span_B': span_B,
        'category_aspect': category_aspect,
        'sentiment': sentiment
    }

def apply_deep_parsing(raw_quadruples_list):
    return [parse_target_quadruples(q) for q in raw_quadruples_list]

def parse_full_line(full_line_string):
    parts = full_line_string.split('\t', 1) 
    if len(parts) != 2:
        return full_line_string.strip(), []

    review_text = parts[0].strip()
    target_string_raw = parts[1].strip()
    
    # Pulizia residui numerici a fine riga
    target_string_clean = re.sub(r'[\r\n\s]\d+$', '', target_string_raw).strip()
    raw_quadruples_list = [q.strip() for q in target_string_clean.split('\t') if q.strip()]

    return review_text, raw_quadruples_list

def apply_full_parsing_pipeline(df):
    """
    Applica la pipeline di parsing restituendo un nuovo DataFrame processato.
    """
    df_res = df.copy() 
    
    # Passaggio 1: Split testo e liste di stringhe target
    parsed_lines = df_res['full_line_data'].apply(parse_full_line)
    df_res['review_text'] = parsed_lines.apply(lambda x: x[0])
    df_res['raw_quadruples_list'] = parsed_lines.apply(lambda x: x[1])
    
    # Passaggio 2: Decodifica delle quadruple in dizionari
    df_res['parsed_quadruples'] = df_res['raw_quadruples_list'].apply(apply_deep_parsing)
    
    # Pulizia colonne temporanee e originali
    return df_res.drop(columns=['full_line_data', 'raw_quadruples_list'])


# --- II. ESECUZIONE CON COPIA DEI DATASET ---

print("Creazione delle copie e avvio del parsing...")

# 1. Creazione delle copie dedicate al parsing
df_train_parsing_laptop = df_train_laptop.copy()
df_dev_parsing_laptop = df_dev_laptop.copy()
df_test_parsing_laptop = df_test_laptop.copy()

df_train_parsing_rest = df_train_rest.copy()
df_dev_parsing_rest = df_dev_rest.copy()
df_test_parsing_rest = df_test_rest.copy()

# 2. Applicazione della pipeline sulle nuove variabili
df_train_parsing_laptop = apply_full_parsing_pipeline(df_train_parsing_laptop)
df_dev_parsing_laptop = apply_full_parsing_pipeline(df_dev_parsing_laptop)
df_test_parsing_laptop = apply_full_parsing_pipeline(df_test_parsing_laptop)

df_train_parsing_rest = apply_full_parsing_pipeline(df_train_parsing_rest)
df_dev_parsing_rest = apply_full_parsing_pipeline(df_dev_parsing_rest)
df_test_parsing_rest = apply_full_parsing_pipeline(df_test_parsing_rest)

print("Parsing completato con successo.")

# --- III. VERIFICA FINALE ---

print("\nAnteprima del DataFrame di Test (df_test_parsing_laptop):")
display(df_test_parsing_laptop[['review_text', 'parsed_quadruples']].head())

print("\nAnteprima del DataFrame di Test (df_test_parsing_rest):")
display(df_test_parsing_rest[['review_text', 'parsed_quadruples']].head())

# Controllo specifico richiesto (riga 3)
print(f"\nQuadruple decodificate: {df_test_parsing_rest['parsed_quadruples'].loc[3]}")
print(f"Testo corrispondente: {df_test_parsing_rest['review_text'].loc[3]}")

print("\nDimensioni finali (Train, Dev, Test):")
print(df_train_parsing_laptop.shape, df_dev_parsing_laptop.shape, df_test_parsing_laptop.shape)
print(df_train_parsing_rest.shape, df_dev_parsing_rest.shape, df_test_parsing_rest.shape)

Creazione delle copie e avvio del parsing...
Parsing completato con successo.

Anteprima del DataFrame di Test (df_test_parsing_laptop):


,review_text,parsed_quadruples
0,"the unit cost $ 275 to start with , so it is n...","[{'span_A': (1, 2), 'span_B': (12, 14), 'categ..."
1,going from ace ##r 15 to ace ##r 11 was diffic...,"[{'span_A': (6, 9), 'span_B': (10, 11), 'categ..."
2,also it ' s not a true ss ##d drive in there b...,"[{'span_A': (7, 10), 'span_B': (-1, -1), 'cate..."
3,the computer has difficulty switching between ...,"[{'span_A': (1, 2), 'span_B': (3, 4), 'categor..."
4,2 / 28 / 18 - a couple days ago i updated the ...,"[{'span_A': (13, 15), 'span_B': (-1, -1), 'cat..."



Anteprima del DataFrame di Test (df_test_parsing_rest):


,review_text,parsed_quadruples
0,yu ##m !,"[{'span_A': (-1, -1), 'span_B': (0, 2), 'categ..."
1,serves really good su ##shi .,"[{'span_A': (3, 5), 'span_B': (2, 3), 'categor..."
2,not the biggest portions but adequate .,"[{'span_A': (3, 4), 'span_B': (0, 3), 'categor..."
3,green tea cr ##eme br ##ule ##e is a must !,"[{'span_A': (0, 7), 'span_B': (9, 10), 'catego..."
4,it has great su ##shi and even better service .,"[{'span_A': (3, 5), 'span_B': (2, 3), 'categor..."



Quadruple decodificate: [{'span_A': (0, 7), 'span_B': (9, 10), 'category_aspect': 'FOOD#QUALITY', 'sentiment': 2}]
Testo corrispondente: green tea cr ##eme br ##ule ##e is a must !

Dimensioni finali (Train, Dev, Test):
(2934, 2) (326, 2) (816, 2)
(1530, 2) (171, 2) (583, 2)


## Sub-word Alignment & BIO Labeling

### Tokenizzazione e Allineamento delle Label
Questo codice serve per convertire i dati dal formato strutturato di Pandas *(review_text e parsed_quadruples)* nel formato numerico accettato dal modello ModernBERT per l'addestramento.

Il processo si svolge in tre passaggi chiave:

1. **Tokenizzazione:** La recensione pulita *(review_text)* viene convertita in una sequenza di *Input IDs* (numeri) utilizzando il tokenizer di ModernBERT.

2. **Allineamento degli Span:** Utilizziamo le funzioni integrate di Hugging Face per mappare gli Indici Span grezzi ((0, 2), etc.) agli indici dei nuovi subword tokens generati dal tokenizer. Questo passaggio risolve i problemi creati dai ## (tokenizzazione subword).

3. **Sequence Labeling (Codifica BIO):** Sulla base degli indici allineati, creiamo l'array finale di Label Numeriche per ogni token della recensione (es. B-ASPECT, I-OPINION, O).

Questo output (Input IDs e Label Sequence) è la forma finale del dataset, pronto per essere passato al Trainer di ModernBERT per il fine-tuning.

In [6]:
from transformers import AutoTokenizer

# 1. Caricamento del Tokenizer di ModernBERT
tokenizer_name = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

def encode_and_align_labels(words, parsed_quadruples, tokenizer, max_len=128):
    """
    Tokenizza il testo e allinea gli span BIO in modo che solo il primo token 
    di uno span sia 'B-' e i successivi siano 'I-'.
    """
    tokenized_input = tokenizer(
        words,
        truncation=True,
        max_length=max_len,
        padding='max_length',
        is_split_into_words=True
    )

    labels = ["O"] * max_len
    word_ids = tokenized_input.word_ids()
    
    for quad in parsed_quadruples:
        # Estraiamo gli span per Aspect (A) e Opinion (B)
        asp_span = quad.get('span_A', (-1, -1))
        opi_span = quad.get('span_B', (-1, -1))
        
        # Funzione di utilità per allineare uno span specifico
        def align_span(span, b_tag, i_tag):
            if span == (-1, -1):
                return
            
            start_word_idx, end_word_idx = span
            span_started = False # Flag per gestire il passaggio da B a I
            
            for i, word_id in enumerate(word_ids):
                if i >= max_len or word_id is None:
                    continue
                
                # Se il token appartiene a una parola compresa nello span [start, end)
                if start_word_idx <= word_id < end_word_idx:
                    if not span_started:
                        labels[i] = b_tag
                        span_started = True
                    else:
                        labels[i] = i_tag

        # Applichiamo la logica a entrambi gli span
        align_span(asp_span, "B-ASP", "I-ASP")
        align_span(opi_span, "B-OPI", "I-OPI")

    # Mappatura in ID numerici
    label_map = {"O": 0, "B-ASP": 1, "I-ASP": 2, "B-OPI": 3, "I-OPI": 4}
    label_ids = [label_map[l] for l in labels]

    return {
        'input_ids': tokenized_input['input_ids'],
        'attention_mask': tokenized_input['attention_mask'],
        'labels': label_ids
    }
    


def process_align_dataset(df, tokenizer, split_name):
    """
    Applica l'allineamento e aggiunge le colonne input_ids, attention_mask e labels.
    """
    print(f"Allineamento in corso per lo split: {split_name}...")
    
    # Applichiamo la funzione riga per riga
    # Usiamo .split() perché il testo è pre-tokenizzato nel dataset originale
    processed_series = df.apply(
        lambda row: encode_and_align_labels(
            row['review_text'].split(), 
            row['parsed_quadruples'], 
            tokenizer, 
            max_len=128
        ), 
        axis=1
    )
    
    # Convertiamo la lista di dizionari in un DataFrame e lo concateniamo
    df_result = pd.DataFrame(processed_series.tolist(), index=df.index)
    return pd.concat([df, df_result], axis=1)

import pandas as pd

# --- 1. CREAZIONE DELLE COPIE DEI DATASET ---
# Creiamo copie profonde per evitare di modificare i DataFrame originali
df_train_align_laptop = df_train_parsing_laptop.copy()
df_dev_align_laptop = df_dev_parsing_laptop.copy()
df_test_align_laptop = df_test_parsing_laptop.copy()

df_train_align_rest = df_train_parsing_rest.copy()
df_dev_align_rest = df_dev_parsing_rest.copy()
df_test_align_rest = df_test_parsing_rest.copy()

# --- 2. ESECUZIONE DELL'ALLINEAMENTO ---
df_train_align_laptop = process_align_dataset(df_train_align_laptop, tokenizer, "TRAIN_LAPTOP")
df_dev_align_laptop = process_align_dataset(df_dev_align_laptop, tokenizer, "DEV_LAPTOP")
df_test_align_laptop = process_align_dataset(df_test_align_laptop, tokenizer, "TEST_LAPTOP")

df_train_align_rest = process_align_dataset(df_train_align_rest, tokenizer, "TRAIN_RESTAURANT")
df_dev_align_rest = process_align_dataset(df_dev_align_rest, tokenizer, "DEV_RESTAURANT")
df_test_align_rest = process_align_dataset(df_test_align_rest, tokenizer, "TEST_RESTAURANT")

# --- 3. OUTPUT DELLE PRIME RIGHE ---
print("\n" + "="*50)
print("VISUALIZZAZIONE DEI DATASET ALLINEATI")

print("\n--- PRIME RIGHE TRAIN LAPTOP---")
display(df_train_align_laptop[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE DEV LAPTOP ---")
display(df_dev_align_laptop[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE TEST LAPTOP ---")
display(df_test_align_laptop[['review_text', 'input_ids', 'labels']].head())


print("\n--- PRIME RIGHE TRAIN RESTAURANT---")
display(df_train_align_rest[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE DEV RESTAURANT ---")
display(df_dev_align_rest[['review_text', 'input_ids', 'labels']].head())

print("\n--- PRIME RIGHE TEST RESTAURANT ---")
display(df_test_align_rest[['review_text', 'input_ids', 'labels']].head())


Allineamento in corso per lo split: TRAIN_LAPTOP...
Allineamento in corso per lo split: DEV_LAPTOP...
Allineamento in corso per lo split: TEST_LAPTOP...
Allineamento in corso per lo split: TRAIN_RESTAURANT...
Allineamento in corso per lo split: DEV_RESTAURANT...
Allineamento in corso per lo split: TEST_RESTAURANT...

VISUALIZZAZIONE DEI DATASET ALLINEATI

--- PRIME RIGHE TRAIN LAPTOP---


,review_text,input_ids,labels
0,ace ##r wants $ 170 to just look at it then ad...,"[50281, 584, 817, 83, 88, 1103, 5, 15046, 936,...","[0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,update : i repaired it myself for $ 12 .,"[50281, 11183, 27, 74, 4762, 12260, 262, 17089...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,i had nothing to lose since it was a paper wei...,"[50281, 74, 10178, 26142, 936, 77, 583, 17480,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,the shame of it is knowing it took me 15 minut...,"[50281, 783, 1200, 482, 1171, 262, 261, 14428,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,first one that they shipped was obviously defe...,"[50281, 7053, 531, 3529, 9328, 1200, 6390, 423...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, ..."



--- PRIME RIGHE DEV LAPTOP ---


,review_text,input_ids,labels
0,this unit is ` ` pretty ` ` and st ##yl ##ish ...,"[50281, 2520, 8522, 261, 65, 65, 38256, 65, 65...","[0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 3, 4, 4, 4, 4, ..."
1,for now i ' m okay with up ##ping the experien...,"[50281, 1542, 2666, 74, 8, 78, 536, 333, 3113,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"seems unlikely but whatever , i ' ll go with it .","[50281, 339, 3030, 328, 10355, 2858, 38499, 13...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,this version has been my least favorite versio...,"[50281, 2520, 4149, 7110, 20394, 2577, 38462, ...","[0, 0, 0, 0, 0, 0, 3, 4, 4, 1, 0, 0, 0, 0, 0, ..."
4,- biggest disappointment is the track pad .,"[50281, 14, 2760, 3219, 3431, 9626, 420, 261, ...","[0, 0, 0, 0, 3, 4, 4, 0, 0, 1, 2, 0, 0, 0, 0, ..."



--- PRIME RIGHE TEST LAPTOP ---


...

,review_text,input_ids,labels
0,judging from previous posts this used to be a ...,"[50281, 6881, 3390, 4064, 35065, 28361, 2520, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,"we , there were four of us , arrived at noon -...","[50281, 664, 13, 9088, 12796, 12496, 1171, 316...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,they never brought us compliment ##ary noodles...,"[50281, 9328, 7594, 1288, 1224, 316, 21013, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,the food was lou ##sy - too sweet or too salty...,"[50281, 783, 19480, 4238, 77, 276, 817, 19089,...","[0, 0, 1, 0, 3, 4, 4, 4, 0, 3, 4, 0, 3, 4, 4, ..."
4,"after all that , they complained to me about t...","[50281, 6438, 455, 3529, 13, 9328, 21013, 1243...","[0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, ..."



--- PRIME RIGHE DEV RESTAURANT ---


,review_text,input_ids,labels
0,ca n ' t wait wait for my next visit .,"[50281, 6357, 79, 8, 85, 14061, 14061, 1542, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"their sake list was extensive , but we were lo...","[50281, 14094, 84, 640, 3550, 4238, 2068, 3134...","[0, 0, 1, 2, 2, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, ..."
2,the spicy tuna roll was unusually good and the...,"[50281, 783, 1033, 2576, 85, 9821, 1811, 4238,...","[0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 3, 0, 0, 1, 2, ..."
3,we love th pink pony .,"[50281, 664, 26617, 394, 49723, 81, 2421, 15, ...","[0, 0, 3, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,this place has got to be the best japanese res...,"[50281, 2520, 5070, 7110, 19559, 936, 1257, 78...","[0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, ..."



--- PRIME RIGHE TEST RESTAURANT ---


,review_text,input_ids,labels
0,yu ##m !,"[50281, 30838, 817, 78, 2, 50282, 50283, 50283...","[0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,serves really good su ##shi .,"[50281, 1498, 265, 28235, 12311, 3467, 817, 41...","[0, 0, 0, 0, 3, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, ..."
2,not the biggest portions but adequate .,"[50281, 1439, 783, 2760, 3219, 631, 621, 2858,...","[0, 3, 4, 4, 4, 1, 2, 0, 3, 4, 0, 0, 0, 0, 0, ..."
3,green tea cr ##eme br ##ule ##e is a must !,"[50281, 11707, 442, 66, 7083, 817, 20867, 1288...","[0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, ..."
4,it has great su ##shi and even better service .,"[50281, 262, 7110, 17124, 3467, 817, 41386, 39...","[0, 0, 0, 3, 1, 2, 2, 0, 0, 3, 1, 0, 0, 0, 0, ..."


## PyTorch Dataset & DataLoader Construction

### Creazione di PyTorch Dataset e DataLoader
In questa fase, trasformiamo i nostri DataFrame Pandas (strutture dati tabellari) in oggetti Dataset e DataLoader di PyTorch. Questo passaggio è il "ponte" necessario per alimentare il modello ModernBERT durante l'addestramento.

#### Obiettivi di questa sezione:

  1. Standardizzazione dei Dati (ACOSDataset):

       * I modelli basati su Transformer non possono leggere direttamente i DataFrame. La classe ACOSDataset estrae le liste di input_ids, attention_mask e labels e le converte in Tensori PyTorch (torch.tensor).

       * Viene utilizzato il tipo di dato torch.long, richiesto dai layer di embedding e dalle funzioni di calcolo della Loss per task di classificazione.

  2. Gestione del Caricamento (DataLoader):

      * Batching: Invece di caricare l'intero dataset in memoria (rischioso per la GPU), i dati vengono divisi in piccoli blocchi chiamati Batch (nel nostro caso di dimensione 16).

      * Shuffling (Solo Training): Utilizziamo shuffle=True nel train_loader per rimescolare l'ordine delle frasi a ogni epoca. Questo impedisce al modello di imparare l'ordine sequenziale dei dati, costringendolo invece a focalizzarsi sui pattern linguistici reali.

      * Efficienza: I DataLoader gestiscono il caricamento dei dati in parallelo, ottimizzando i tempi di addestramento sulla GPU.

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class ACOSDataset(Dataset):
    def __init__(self, df):
        # Estraiamo le colonne che abbiamo generato nella fase di allineamento
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['labels'].tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        # Convertiamo le liste in Tensori di PyTorch (LongTensor per ID e Label)
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# --- CREAZIONE DELLE ISTANZE ---

# Creiamo i dataset per il dominio Laptop
train_dataset_laptop = ACOSDataset(df_train_align_laptop)
dev_dataset_laptop = ACOSDataset(df_dev_align_laptop)
test_dataset_laptop = ACOSDataset(df_test_align_laptop)

# Creiamo i dataset per il dominio restaruant
train_dataset_rest = ACOSDataset(df_train_align_rest)
dev_dataset_rest = ACOSDataset(df_dev_align_rest)
test_dataset_rest = ACOSDataset(df_test_align_rest)

# --- CONFIGURAZIONE DATALOADERS ---

BATCH_SIZE = 16 # Numero di frasi analizzate contemporaneamente

train_loader_laptop = DataLoader(train_dataset_laptop, batch_size=BATCH_SIZE, shuffle=True)
dev_loader_laptop = DataLoader(dev_dataset_laptop, batch_size=BATCH_SIZE)
test_loader_laptop = DataLoader(test_dataset_laptop, batch_size=BATCH_SIZE)

train_loader_rest = DataLoader(train_dataset_rest, batch_size=BATCH_SIZE, shuffle=True)
dev_loader_rest = DataLoader(dev_dataset_rest, batch_size=BATCH_SIZE)
test_loader_rest = DataLoader(test_dataset_rest, batch_size=BATCH_SIZE)

print(f"Dataset e DataLoaders creati con successo!")
print(f"Esempi nel set di Training LAPTOP: {len(train_dataset_laptop)}")
print(f"Esempi nel set di Training RESTAURANT: {len(train_dataset_rest)}")  

Dataset e DataLoaders creati con successo!
Esempi nel set di Training LAPTOP: 2934
Esempi nel set di Training RESTAURANT: 1530


### Definizione e l'Inizializzazione del Modello di Token Classification.

1. Caricare il "Cervello" (ModernBERT Pre-trained)
Dobbiamo scaricare il modello ModernBERT-base dal repository di Hugging Face. In questa fase, il modello sa già "leggere" e "capire" la lingua inglese perché è stato addestrato su miliardi di testi, ma non sa ancora nulla del tuo task specifico (ACOS). È come un laureato in lingue che però non ha mai lavorato in un ristorante o in un negozio di computer.

2. Aggiungere la "Testa" di Classificazione
ModernBERT normalmente restituisce dei vettori numerici (embedding) per ogni parola. Noi dobbiamo aggiungere sopra questi vettori uno strato finale chiamato Linear Layer (o testa di classificazione).

   * Questo strato prenderà l'output di ModernBERT e lo "schiaccerà" su 5 classi possibili: 0 (O), 1 (B-ASP), 2 (I-ASP), 3 (B-OPI), 4 (I-OPI).

   * Il modello dovrà imparare a mappare ogni pezzetto di frase a una di queste cinque etichette.

3. Configurare la Strategia di Apprendimento (Optimizer & Loss)
Dobbiamo dare al modello gli strumenti per imparare dai suoi errori:

  * Loss Function (Funzione di Perdita): Useremo la CrossEntropyLoss. È il "voto" che diamo al modello. Se il modello dice che "pizza" è un'opinione (B-OPI) ma il tuo dataset dice che è un aspetto (B-ASP), la Loss sarà alta. Il modello cercherà di abbassarla il più possibile.

   * Optimizer (Ottimizzatore): Di solito si usa AdamW. È l'algoritmo che decide "come" e "quanto" cambiare i pesi interni del modello per correggere gli errori.

   * Learning Rate: La velocità con cui il modello impara. Se è troppo alta, il modello è "frettoloso" e sbaglia; se è troppo bassa, non imparerà mai.

In [8]:
import torch
from torch.optim import AdamW
from transformers import AutoModelForTokenClassification
import torch.nn as nn

# --- 1. CONFIGURAZIONE DEL DEVICE ---
# Se hai una GPU NVIDIA, userà 'cuda'. Se hai un Mac M1/M2, userà 'mps'. Altrimenti 'cpu'.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f" GPU Trovata: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(" Acceleratore Apple Metal (MPS) Trovato")
else:
    device = torch.device("cpu")
    print(" Nessuna GPU trovata. L'addestramento sarà lento.")

# --- 2. CARICAMENTO DI MODERNBERT (IL "CERVELLO") + TESTA DI CLASSIFICAZIONE ---
# Definiamo le 5 etichette: 0=O, 1=B-ASP, 2=I-ASP, 3=B-OPI, 4=I-OPI
NUM_LABELS = 5 

print("Scaricamento e configurazione di ModernBERT...")
model = AutoModelForTokenClassification.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=NUM_LABELS
)

# Spostiamo il modello sul dispositivo di calcolo (GPU/MPS/CPU)
model.to(device)

# --- 3. CONFIGURAZIONE DELL'OTTIMIZZATORE E DELLA LOSS ---

# A. Optimizer (AdamW)
# Usiamo il Learning Rate standard di 5e-5 come definito nei parametri sperimentali 
optimizer = AdamW(model.parameters(), lr=5e-5)

# B. Loss Function (CrossEntropyLoss)
# La funzione che calcola l'errore tra la predizione del modello e le label reali.
# Nota: 'ignore_index=-100' è lo standard di PyTorch per ignorare i token di padding nel calcolo dell'errore.
loss_fn = nn.CrossEntropyLoss()

print("\n" + "="*50)
print("MODELLO PRONTO PER IL TRAINING")
print("="*50)
print(f"Architettura: ModernBERT-base")
print(f"Task: Token Classification (Estrazione Aspetti & Opinioni)")
print(f"Numero di Classi: {NUM_LABELS}")
print(f"Optimizer: AdamW (lr=5e-5)")
print(f"Loss Function: CrossEntropyLoss")

 Acceleratore Apple Metal (MPS) Trovato
Scaricamento e configurazione di ModernBERT...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading weights:   0%|          | 0/136 [00:00<?, ?it/s]

ModernBertForTokenClassification LOAD REPORT from: answerdotai/ModernBERT-base
Key               | Status     | 
------------------+------------+-
decoder.bias      | UNEXPECTED | 
classifier.bias   | MISSING    | 
classifier.weight | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.



MODELLO PRONTO PER IL TRAINING
Architettura: ModernBERT-base
Task: Token Classification (Estrazione Aspetti & Opinioni)
Numero di Classi: 5
Optimizer: AdamW (lr=5e-5)
Loss Function: CrossEntropyLoss


In [9]:
import torch
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
import wandb # Assumo che tu abbia già importato wandb

# --- CONFIGURAZIONE EARLY STOPPING ---
# Impostiamo la pazienza: quante epoche aspettare senza miglioramenti?
patience = config.get('patience', 2)  # Prende da config o usa 3 come default
patience_counter = 0                  # Contatore inizializzato a 0

# L'ottimizzatore prende il learning rate dalla tua variabile 'config'
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=config['learning_rate'] 
)

# Scheduler: Riduce il learning rate linearmente
total_steps = len(train_loader_laptop) * config['epochs']
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

# --- 2. FUNZIONI DI SUPPORTO ---

def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()
    return total_loss / len(data_loader)

def train_epoch(model, data_loader, optimizer, scheduler, device, epoch_idx):
    model.train()
    total_loss = 0
    loop = tqdm(data_loader, leave=True)
    
    for batch in loop:
        # A. Dati su GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # B. Reset e Calcoli
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # C. Aggiornamento Pesi
        loss.backward()
        optimizer.step()
        scheduler.step() 
        
        total_loss += loss.item()
        
        # Log istantaneo su W&B (opzionale, per non intasare i grafici)
        wandb.log({"batch_loss": loss.item()})
        
        loop.set_description(f"Epoca {epoch_idx + 1}")
        loop.set_postfix(loss=loss.item())

    return total_loss / len(data_loader)

# --- 3. CICLO DI ADDESTRAMENTO (LOOP) ---

print(f"🚀 Inizio Training su LAPTOP: {config['epochs']} epoche su {device}")
print(f"🛑 Early Stopping configurato con Patience = {patience}")

best_valid_loss_laptop = float('inf')

for epoch in range(config['epochs']):
    print(f"\n--- Epoca {epoch+1}/{config['epochs']} ---")
    
    # 1. Training
    train_loss_laptop = train_epoch(model, train_loader_laptop, optimizer, scheduler, device, epoch)
    
    # 2. Validazione
    valid_loss_laptop = evaluate_model(model, dev_loader_laptop, device)
    
    print(f"📉 Train Loss: {train_loss_laptop:.4f} | 🔍 Valid Loss: {valid_loss_laptop:.4f}")
    
    # 3. Log metriche epoca su W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss_epoch": train_loss_laptop,
        "valid_loss_epoch": valid_loss_laptop
    })
    
    # --- LOGICA EARLY STOPPING & CHECKPOINT ---
    
    # Se il modello migliora (la valid loss scende)
    if valid_loss_laptop < best_valid_loss_laptop:
        best_valid_loss_laptop = valid_loss_laptop
        patience_counter = 0  # ### NUOVO: Resettiamo la pazienza ###
        
        print(f"💾 Miglior modello trovato (Loss: {best_valid_loss_laptop:.4f})! Salvataggio...")
        
        # Creiamo la cartella se non esiste (sicurezza aggiuntiva)
        import os
        if not os.path.exists("./best_model_laptop"):
            os.makedirs("./best_model_laptop")
            
        model.save_pretrained("./best_model_laptop")
        
    # Se il modello NON migliora
    else:
        patience_counter += 1  # ### NUOVO: Incrementiamo il contatore ###
        print(f"⚠️ Nessun miglioramento. Patience: {patience_counter}/{patience}")
        
        # Se abbiamo esaurito la pazienza
        if patience_counter >= patience:
            print(f"\n🛑 EARLY STOPPING ATTIVATO! Interruzione all'epoca {epoch+1}.")
            break # Esce dal ciclo for

print("\n✅ Fine Addestramento.")
wandb.finish()

🚀 Inizio Training su LAPTOP: 5 epoche su mps
🛑 Early Stopping configurato con Patience = 2

--- Epoca 1/5 ---


  0%|          | 0/184 [00:00<?, ?it/s]

Epoca 1: 100%|██████████| 184/184 [1:01:52<00:00, 20.17s/it, loss=0.0488]


📉 Train Loss: 0.0710 | 🔍 Valid Loss: 0.0343
💾 Miglior modello trovato (Loss: 0.0343)! Salvataggio...


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoca 4: 100%|██████████| 184/184 [1:08:28<00:00, 22.33s/it, loss=0.00218]


📉 Train Loss: 0.0098 | 🔍 Valid Loss: 0.0330
⚠️ Nessun miglioramento. Patience: 1/2

--- Epoca 5/5 ---


Epoca 5: 100%|██████████| 184/184 [1:19:17<00:00, 25.86s/it, loss=0.0117] 


📉 Train Loss: 0.0048 | 🔍 Valid Loss: 0.0340
⚠️ Nessun miglioramento. Patience: 2/2

🛑 EARLY STOPPING ATTIVATO! Interruzione all'epoca 5.

✅ Fine Addestramento.


batch_loss,█▅▅▄▂▃▂▃▂▃▃▂▂▃▃▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▃▁▁▁▁▁▁
epoch,▁▃▅▆█
train_loss_epoch,█▃▂▂▁
valid_loss_epoch,█▄▁▆█
batch_loss,0.01166
epoch,5
train_loss_epoch,0.00477
valid_loss_epoch,0.03401
